# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the CSV file
    dataframe = pd.read_csv('../../data/logs/medium_log.csv', sep=',')

    # Format the dataframe
    dataframe = pm4py.format_dataframe(
        dataframe,
        case_id='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_6451/51279120.py:11: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe = pm4py.format_dataframe(


# Data Preprocessing

## Integer Encoding 

In [2]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
dataframe['activity_encoded'] = label_encoder.fit_transform(dataframe['concept:name'])

# Now the 'dataframe' has a new column 'activity_encoded' with integer encoded values of the 'activity' column
dataframe

,concept:name,timestamp,time:timestamp,case:label,case:concept:name,org:resource,@@index,@@case_index,activity_encoded
0,Activity A,NaN,2010-01-01 00:00:00+00:00,Insert,1,Doretta,0,0,0
1,Activity C,NaN,2010-01-01 00:01:00+00:00,Insert,1,Jack,1,0,8
2,Activity F,NaN,2010-01-01 00:02:00+00:00,Insert,1,Howard,2,0,11
3,Random activity 31,NaN,2010-01-01 00:04:00+00:00,Insert,1,Random user 69,3,0,56
4,Activity D,NaN,2010-01-01 00:05:00+00:00,Insert,1,Wilton,4,0,9
...,...,...,...,...,...,...,...,...,...
31986,Activity D,NaN,2010-07-01 09:37:00+00:00,normal,999,Wilton,31986,4999,9
31987,Activity I,NaN,2010-07-01 09:38:00+00:00,normal,999,Alyce,31987,4999,14
31988,Activity L,NaN,2010-07-01 09:40:00+00:00,normal,999,Rossana,31988,4999,17
31989,Activity J,NaN,2010-07-01 09:41:00+00:00,normal,999,Jamika,31989,4999,15


# Build Input X and output Y

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    
    return windows_padded, targets_array

2024-08-11 21:11:16.323229: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
activity = dataframe[['activity_encoded', '@@case_index']]

In [5]:
X, Y = generate_prefix_windows(activity)

# LSTM Model Architecture

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Lambda

# Define parameters
input_dim = dataframe['concept:name'].nunique()  # Number of unique activities
embedding_dim = len(dataframe['concept:name'].unique())  # Embedding dimension
lstm_units = 100  # Number of units in the LSTM layer
dropout_rate = 0.2  # Dropout rate to prevent overfitting
output_dim = input_dim  # Output dimension (number of activities)

# Define the input layer
inputs = Input(shape=(None,))

# Embedding layer
embedding = Embedding(input_dim=input_dim, output_dim=embedding_dim)(inputs)

# LSTM layer with return_sequences=True to get hidden states at each time step
lstm_output, lstm_state_h, lstm_state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(embedding)

# Apply a Lambda layer to extract the last hidden state
last_hidden_state = Lambda(lambda x: x[:, -1, :])(lstm_output)

# Dropout layer
dropout = Dropout(rate=dropout_rate)(last_hidden_state)

# Dense output layer for next activity prediction
outputs = Dense(units=output_dim, activation='softmax')(dropout)

# Build the model for training (only outputs the prediction)
training_model = Model(inputs=inputs, outputs=outputs)

# Compile the training model
training_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model for inference (outputs both predictions and hidden states)
inference_model = Model(inputs=inputs, outputs=[outputs, lstm_output])

# Display the model architecture
training_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 63)          3969      
                                                                 
 lstm (LSTM)                 [(None, None, 100),       65600     
                              (None, 100),                       
                              (None, 100)]                       
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 63)                6363  

# LSTM Training

In [7]:
from tensorflow.keras.utils import to_categorical

Y = to_categorical(Y, num_classes=input_dim)

In [8]:
history = training_model.fit(X, Y, epochs=25, batch_size=500)

Epoch 1/25
54/54 [==============================] - 4s 38ms/step - loss: 3.3462 - accuracy: 0.1709
Epoch 2/25
54/54 [==============================] - 2s 38ms/step - loss: 2.8657 - accuracy: 0.1968
Epoch 3/25
54/54 [==============================] - 2s 41ms/step - loss: 2.1914 - accuracy: 0.3534
Epoch 4/25
54/54 [==============================] - 2s 38ms/step - loss: 1.5266 - accuracy: 0.5806
Epoch 5/25
54/54 [==============================] - 2s 39ms/step - loss: 1.2297 - accuracy: 0.6603
Epoch 6/25
54/54 [==============================] - 2s 39ms/step - loss: 1.1122 - accuracy: 0.6809
Epoch 7/25
54/54 [==============================] - 2s 42ms/step - loss: 1.0454 - accuracy: 0.6894
Epoch 8/25
54/54 [==============================] - 2s 38ms/step - loss: 0.9919 - accuracy: 0.6943
Epoch 9/25
54/54 [==============================] - 2s 39ms/step - loss: 0.9585 - accuracy: 0.6992
Epoch 10/25
54/54 [==============================] - 2s 41ms/step - loss: 0.9274 - accuracy: 0.7010
Epoch 11/

# Extract Trace Representations Using LSTM Network

In [9]:
predictions, hidden_states = inference_model.predict(X)

844/844 [==============================] - 6s 6ms/step


In [10]:
# Group by case index to get individual traces
traces = dataframe.groupby('@@case_index')['concept:name'].apply(list)
traces

@@case_index
0       [Activity A, Activity C, Activity F, Random ac...
1                    [Activity A, Activity Y, Activity B]
2                    [Activity Y, Activity A, Activity B]
3       [Activity C, Activity A, Activity F, Activity ...
4       [Activity A, Activity O, Activity U, Activity ...
                              ...                        
4995    [Activity A, Activity C, Activity G, Activity ...
4996                 [Activity Y, Activity A, Activity B]
4997    [Activity A, Activity O, Activity S, Activity ...
4998    [Activity A, Activity C, Activity G, Activity ...
4999    [Activity A, Activity C, Activity F, Activity ...
Name: concept:name, Length: 5000, dtype: object

In [11]:
hidden_state_pos = traces.apply(len).values
hidden_state_pos = hidden_state_pos - 1
hidden_state_pos

array([9, 2, 2, ..., 7, 7, 7])

In [12]:
trace_representations = [hidden_states[i-1] for i in hidden_state_pos]
trace_representations = np.array(trace_representations)
len(trace_representations)

5000

In [13]:
trace_representations.shape

(5000, 10, 100)

# Ground Truth

In [14]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [15]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/medium.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/689 [00:00<?, ?it/s]

In [16]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [17]:
classification = pd.DataFrame(conformance, columns=['actual'])

# Input Traces

In [18]:
# Step 2: Group by case_index and concatenate the activities to form traces
dataframe['trace'] = dataframe.groupby('@@case_index')['concept:name'].transform(lambda x: ', '.join(x))

# Step 3: Count occurrences of each unique trace
trace_counts = dataframe['trace'].value_counts()

# Step 4: Convert to DataFrame and sort by occurrences
trace_counts_df = trace_counts.reset_index()
trace_counts_df.columns = ['Trace', 'Count']
trace_counts_df = trace_counts_df.sort_values(by='Count', ascending=False)


In [19]:
top_3_traces = trace_counts_df.head(20)

trace1 = top_3_traces.iloc[0]['Trace']
trace1 = [event.strip() for event in trace1.split(',')]

trace2 = top_3_traces.iloc[1]['Trace']
trace2 = [event.strip() for event in trace2.split(',')]

trace3 = top_3_traces.iloc[2]['Trace']
trace3 = [event.strip() for event in trace3.split(',')]

trace4 = top_3_traces.iloc[3]['Trace']
trace4 = [event.strip() for event in trace4.split(',')]

trace5 = top_3_traces.iloc[4]['Trace']
trace5 = [event.strip() for event in trace5.split(',')]

trace6 = top_3_traces.iloc[5]['Trace']
trace6 = [event.strip() for event in trace6.split(',')]

trace7 = top_3_traces.iloc[6]['Trace']
trace7 = [event.strip() for event in trace7.split(',')]

trace8 = top_3_traces.iloc[7]['Trace']
trace8 = [event.strip() for event in trace8.split(',')]

trace9 = top_3_traces.iloc[8]['Trace']
trace9 = [event.strip() for event in trace9.split(',')]

trace10 = top_3_traces.iloc[9]['Trace']
trace10 = [event.strip() for event in trace10.split(',')]

trace11 = top_3_traces.iloc[10]['Trace']
trace11 = [event.strip() for event in trace11.split(',')]

trace12 = top_3_traces.iloc[11]['Trace']
trace12 = [event.strip() for event in trace12.split(',')]

trace13 = top_3_traces.iloc[12]['Trace']
trace13 = [event.strip() for event in trace13.split(',')]

trace14 = top_3_traces.iloc[13]['Trace']
trace14 = [event.strip() for event in trace14.split(',')]

trace15 = top_3_traces.iloc[14]['Trace']
trace15 = [event.strip() for event in trace15.split(',')]

In [20]:
grouped = dataframe.groupby('@@case_index')['concept:name'].apply(list).reset_index(name='trace')

In [21]:
def is_happy_trace(row_trace):
    predefined_traces = [trace1, trace2, trace3]
    for trace in predefined_traces:
        if row_trace == trace:
            return 1
    return 0

In [22]:
grouped['happy'] = grouped['trace'].apply(is_happy_trace)
grouped

,@@case_index,trace,happy
0,0,"[Activity A, Activity C, Activity F, Random ac...",0
1,1,"[Activity A, Activity Y, Activity B]",1
2,2,"[Activity Y, Activity A, Activity B]",0
3,3,"[Activity C, Activity A, Activity F, Activity ...",0
4,4,"[Activity A, Activity O, Activity U, Activity ...",1
...,...,...,...
4995,4995,"[Activity A, Activity C, Activity G, Activity ...",0
4996,4996,"[Activity Y, Activity A, Activity B]",0
4997,4997,"[Activity A, Activity O, Activity S, Activity ...",0
4998,4998,"[Activity A, Activity C, Activity G, Activity ...",1


In [23]:
# Get indices of the happy traces in the results dataframe
happy_trace_indices = grouped[grouped['happy'] == 1].index.tolist()

# Extract the corresponding coordinates from the trace_representations array
happy_trace_coordinates = trace_representations[happy_trace_indices]

# Extract unique coordinates
unique_happy_trace_coordinates = np.unique(happy_trace_coordinates, axis=0)

# Assuming the size of unique_happy_trace_coordinates is 3
#happy_trace1, happy_trace2, happy_trace3 = unique_happy_trace_coordinates
happy_trace1, happy_trace2, happy_trace3 = unique_happy_trace_coordinates

# Distance Measurement

In [24]:
# Flatten each trace representation (7, 100) into a 1D vector of size 700
trace_representations = trace_representations.reshape(trace_representations.shape[0], -1)

# Now flattened_trace_representations will have shape (13087, 700)
happy_trace1 = happy_trace1.flatten()
happy_trace2 = happy_trace2.flatten()
happy_trace3 = happy_trace3.flatten()

In [25]:
happy_trace1.shape

(1000,)

In [26]:
trace_representations[0].shape

(1000,)

In [27]:
from scipy.spatial.distance import euclidean

# Calculate the distances to each of the happy traces for every trace representation
distances_to_happy_traces = []

for trace_representation in trace_representations:
    distances = [
        euclidean(trace_representation, happy_trace1),
        euclidean(trace_representation, happy_trace2),
        euclidean(trace_representation, happy_trace3)
    ]
    distances_to_happy_traces.append(distances)

# Calculate the average distance to the happy traces for each trace representation
avg_distances = [np.mean(distances) for distances in distances_to_happy_traces]

# Save the distances in a variable
avg_distances_var = np.array(avg_distances)

In [28]:
classification['distance'] = avg_distances_var

# Evaluation

In [29]:
# Determine Threshold

from sklearn.cluster import KMeans

# Filter the DataFrame into conforming and non-conforming subsets
conforming_distances = classification[classification['actual'] == 0]['distance']
non_conforming_distances = classification[classification['actual'] == 1]['distance']

# Determine common bin edges
min_distance = min(classification['distance'])
max_distance = max(classification['distance'])
bin_edges = np.linspace(min_distance, max_distance, num=30)

# Combine the data and reshape for k-means
all_distances = avg_distances_var
all_distances = np.array(all_distances)
all_distances_reshaped = all_distances.reshape(-1, 1)

# Apply k-means clustering
kmeans = KMeans(n_clusters=2, random_state=0).fit(all_distances_reshaped)
kmeans_labels = kmeans.labels_

# Find the threshold as the average of the two cluster centers
threshold_value = np.mean(kmeans.cluster_centers_)

In [30]:
# Assuming 'classification' is a DataFrame and 'threshold_value' is already defined
classification['predicted'] = classification['distance'].apply(lambda x: 1 if x > threshold_value else 0)

In [31]:
# Calculating TP, TN, FP, FN
TP = ((classification['actual'] == 1) & (classification['predicted'] == 1)).sum()
TN = ((classification['actual'] == 0) & (classification['predicted'] == 0)).sum()
FP = ((classification['actual'] == 0) & (classification['predicted'] == 1)).sum()
FN = ((classification['actual'] == 1) & (classification['predicted'] == 0)).sum()

In [32]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.33


In [33]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.65


In [34]:
precision_no_d = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_d:.2f}")

Precision No Dev: 0.85


In [35]:
# Calculate recall
recall_no_d = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_d:.2f}")

Recall No Dev: 0.60


In [36]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(classification['actual'], classification['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.63


# Deviations

### a) identify closest trace

In [37]:
# INPUT TRACE 1



import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/input_traces/medium_trace1.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_input_trace1 = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/689 [00:00<?, ?it/s]

In [38]:
# INPUT TRACE 2



import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/input_traces/medium_trace2.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_input_trace2 = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/689 [00:00<?, ?it/s]

In [39]:
# INPUT TRACE 3



import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/input_traces/medium_trace3.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_input_trace3 = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/689 [00:00<?, ?it/s]

In [40]:
# Calculate Euclidean distance
def euclidean_distance(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)

# Prepare a list to store the results
distance = []

# Iterate through each subarray in one_hot_encoding
for subarray in trace_representations:
    dist_to_trace1 = euclidean_distance(subarray, happy_trace1)
    
    distances = [dist_to_trace1]
    closest_trace_index = np.argmin(distances)
    closest_trace = f'trace_{closest_trace_index + 1}'
    
    distance.append({
        'Distance to Trace 1': dist_to_trace1,
        'Closest Trace': closest_trace
    })

# Create DataFrame
closest_distance = pd.DataFrame(distance)

### b) identify deviations

In [41]:
# Initialize the merged list
aligned_input_trace = []

# Iterate through each row of the fitness dataframe
for index, row in closest_distance.iterrows():
    closest_trace = row['Closest Trace']
    
    # Append the corresponding alignment to the merged list based on the closest trace
    if closest_trace == 'trace_1':
        aligned_input_trace.append(aligned_input_trace1[index])
    elif closest_trace == 'trace_2':
        aligned_input_trace.append(aligned_input_trace2[index])
    elif closest_trace == 'trace_3':
        aligned_input_trace.append(aligned_input_trace3[index])

In [42]:
# Extract fitness values
aligned_traces_fitness = [trace['fitness'] for trace in aligned_traces]
aligned_input_traces_fitness = [trace['fitness'] for trace in aligned_input_trace]

# Create DataFrame
df_fitness = pd.DataFrame({
    'ground_truth_fit': aligned_traces_fitness,
    'predicted_fit': aligned_input_traces_fitness
})

In [43]:
# Find indices where 'predicted' column has value 1
indices_to_keep = classification[classification['actual'] == 1].index.tolist()

# Filter the lists to keep only the indices where 'predicted' is 1
aligned_input_trace = [aligned_input_trace[i] for i in indices_to_keep]
aligned_traces = [aligned_traces[i] for i in indices_to_keep]

In [44]:
# Function to extract log and model moves excluding (None, >>) and (>>, None)
def extract_moves(alignment):
    log_moves = [move for move in alignment if move[1] == '>>' and move[0] is not None]
    model_moves = [move for move in alignment if move[0] == '>>' and move[1] is not None]
    return log_moves, model_moves

# Initialize counts for moves
total_log_moves = 0
total_no_log_moves = 0
total_model_moves = 0
total_no_model_moves = 0

# Initialize counts for TP, FP, FN, TN
tp_log_moves = 0
fp_log_moves = 0
fn_log_moves = 0
tn_log_moves = 0

tp_model_moves = 0
fp_model_moves = 0
fn_model_moves = 0
tn_model_moves = 0

# Iterate through aligned traces and count moves
for i, aligned_trace in enumerate(aligned_traces):
    log_moves_gt, model_moves_gt = extract_moves(aligned_trace['alignment'])
    total_log_moves += len(log_moves_gt)
    total_no_log_moves += sum(1 for move in aligned_trace['alignment'] if move[1] != '>>' or move[0] is None)
    total_model_moves += len(model_moves_gt)
    total_no_model_moves += sum(1 for move in aligned_trace['alignment'] if move[0] != '>>' or move[1] is None)
    
    if i < len(aligned_input_trace):
        log_moves_input, model_moves_input = extract_moves(aligned_input_trace[i]['alignment'])
        
        # Calculate TP, FP, FN, TN for log moves
        tp_log_moves += sum(1 for move in log_moves_gt if move in log_moves_input)
        fn_log_moves += sum(1 for move in log_moves_gt if move not in log_moves_input)
        fp_log_moves += sum(1 for move in log_moves_input if move not in log_moves_gt)
        tn_log_moves += sum(1 for move in aligned_trace['alignment'] if move not in log_moves_gt and move not in log_moves_input and move[1] != '>>' and move[0] != '>>')
        
        # Calculate TP, FP, FN, TN for model moves
        tp_model_moves += sum(1 for move in model_moves_gt if move in model_moves_input)
        fn_model_moves += sum(1 for move in model_moves_gt if move not in model_moves_input)
        fp_model_moves += sum(1 for move in model_moves_input if move not in model_moves_gt)
        tn_model_moves += sum(1 for move in aligned_trace['alignment'] if move not in model_moves_gt and move not in model_moves_input and move[1] != '>>' and move[0] != '>>')

# Calculate recall, precision, F1 score for log moves
recall_log_moves = tp_log_moves / (tp_log_moves + fn_log_moves) if (tp_log_moves + fn_log_moves) > 0 else 0
precision_log_moves = tp_log_moves / (tp_log_moves + fp_log_moves) if (tp_log_moves + fp_log_moves) > 0 else 0
f1_score_log_moves = 2 * (precision_log_moves * recall_log_moves) / (precision_log_moves + recall_log_moves) if (precision_log_moves + recall_log_moves) > 0 else 0

# Calculate recall, precision, F1 score for model moves
recall_model_moves = tp_model_moves / (tp_model_moves + fn_model_moves) if (tp_model_moves + fn_model_moves) > 0 else 0
precision_model_moves = tp_model_moves / (tp_model_moves + fp_model_moves) if (tp_model_moves + fp_model_moves) > 0 else 0
f1_score_model_moves = 2 * (precision_model_moves * recall_model_moves) / (precision_model_moves + recall_model_moves) if (precision_model_moves + recall_model_moves) > 0 else 0

# Calculate dataset balance for log moves
log_move_percentage = (total_log_moves / (total_log_moves + total_no_log_moves)) * 100 if (total_log_moves + total_no_log_moves) > 0 else 0
no_log_move_percentage = (total_no_log_moves / (total_log_moves + total_no_log_moves)) * 100 if (total_log_moves + total_no_log_moves) > 0 else 0

# Calculate dataset balance for model moves
model_move_percentage = (total_model_moves / (total_model_moves + total_no_model_moves)) * 100 if (total_model_moves + total_no_model_moves) > 0 else 0
no_model_move_percentage = (total_no_model_moves / (total_model_moves + total_no_model_moves)) * 100 if (total_model_moves + total_no_model_moves) > 0 else 0

# Print results for log moves
print(f"Precision (Log Moves): {precision_log_moves:.2f}")
print(f"Recall (Log Moves): {recall_log_moves:.2f}")
print("")

# Print results for model moves
print(f"Precision (Model Moves): {precision_model_moves:.2f}")
print(f"Recall (Model Moves): {recall_model_moves:.2f}")

Precision (Log Moves): 0.35
Recall (Log Moves): 0.86

Precision (Model Moves): 0.08
Recall (Model Moves): 0.34


In [45]:
# Filter the other dataframe using the indices_to_keep
df_fitness = df_fitness.loc[indices_to_keep]

In [46]:
from sklearn.metrics import mean_squared_error

# Calculate MSE
mse = mean_squared_error(df_fitness['ground_truth_fit'], df_fitness['predicted_fit'])

# Print the MSE restricted to 4 decimal places
print(f"The Mean Squared Error (MSE) is: {mse:.4f}")

The Mean Squared Error (MSE) is: 0.0462
